# ML Techniques for Time Series Stock Data

# Imports

In [1]:
import math
import numpy as np
import pandas as pd
import yfinance as yf

from abc import ABC
from typing import List
from tensorflow import keras
from abc import abstractmethod
from dataclasses import dataclass


from keras.layers import LSTM as model_LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers.convolutional import Conv1D 
from keras.layers.convolutional import MaxPooling1D

from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

2023-09-26 18:41:58.238837: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Processor - Load the stock data

1. Retrieve the stock data with `data = yf.download(symbol, start="YYYY-MM-DD", end="YYYY-MM-DD")`
2. Specify what column we're seeking to explore to get the univariate time series (uts)
3. Convert the univariate time series (uts) to supervised machine learning (sml)

In [2]:
@dataclass
class YFinanceDataProcessor:
    """Handles fetching and returning data. """
    ticker_symbol: str
    start_date: str
    end_date: str
    
    def __post_init__(self):
        """Operations that are performed after the initialization step."""
        self.data = yf.download(self.ticker_symbol, start=self.start_date, end=self.end_date)
    
    def filter_by_col(self, col_name: str):
        """Returns a filtered univariate dataframe with all the rows.
        
        Parameter:
        col_name -- str (that specifies the column) 
        
        Return:
        -- pd DataFrame (of the univariate ts data)
        """
        univariate_ts_df = self.data.loc[:, [col_name]]
        return univariate_ts_df
    
    def convert_uts_sequence_to_sml_with_pd(self, uts_observations: pd.DataFrame, prior_observations: int, forecasting_step: int):
        """Splits a given UTS into multiple input rows where each input row has a specified number of timestamps and the output is a single timestamp.

        Parameters:
        uts_observations -- pd DataFrame (of UTS data to transform to SML data with size  b rows/length x 1 dimension)
        prior_observations -- py int (of all observations before we get to where we want to start making the predictions)
        forecasting_step -- py int (of how far out to forecast, 1 only the next timestamp, 2 the next two timestamps, ... n the next n timestamps)

        Return:
        -- pd DataFrame (of the sml data)
        """
        observations_df = pd.DataFrame(uts_observations)
        cols = list()
        lag_col_names = []
        
        # print("Input Univariate Time Series:")
        # print(uts_observations, "\nX of size", np.shape(uts_observations))
        # print()

        # input sequence (t-n, ... t-1)
        # name columns for sml df
        for prior_observation in range(prior_observations, 0, -1):
            # print("prior_observation: ", prior_observation)
            
            cols.append(observations_df.shift(prior_observation))
            new_col_name = "t-" + str(prior_observation)
            # print(new_col_name)
            lag_col_names.append(new_col_name)


        # forecast sequence (t, t+1, ... t+n)
        # name columns for sml df
        for i in range(0, forecasting_step):
            cols.append(observations_df.shift(-i))

            new_col_name = "t" 
            if forecasting_step == 1:
                # print(new_col_name)
                lag_col_names.append(new_col_name)

            else:
                if i == 0:
                    lag_col_names.append(new_col_name)
                else:
                    new_col_name = "t+" + str(i)
                    # print(new_col_name)
                    lag_col_names.append(new_col_name)

            # put observation cols together and add column names
            uts_sml_df = pd.concat(cols, axis=1) 
            uts_sml_df.columns=[lag_col_names]
            
            # drop rows with NaN values
            uts_sml_df.dropna(inplace=True)
            sml_df = uts_sml_df.reset_index(drop=True)
            
        return sml_df
    
    def train_test_split(self, uts_sml_df: pd.DataFrame):
        
        # print(uts_sml_df)
        # print("Univariate Time Series as Supervised Machine Learning:")
        # colums to use to make prediction for last col

        X_train = uts_sml_df.iloc[:, :prior_observations]
        # print("X_train: \n", X_train)

        # last column
        y_train = uts_sml_df.iloc[:, -forecasting_step:]
        # print("y_train: \n", y_train)
        
        # Make a prediction for this data
        x_input = uts_sml_df.iloc[-1:, forecasting_step:]
        
        return X_train, y_train, x_input

In [3]:
# Set the start and end dates for the data
symbol = "VOO"
# symbol = "BRK-A"
start_date = "2022-01-01"
end_date = "2023-06-26"

data_processor = YFinanceDataProcessor(symbol, start_date, end_date)
# data_processor.data
uts_observations = data_processor.filter_by_col("Open")
uts_observations

[*********************100%***********************]  1 of 1 completed


,Open
Date,
2022-01-03,437.929993
2022-01-04,440.579987
2022-01-05,438.779999
2022-01-06,430.200012
2022-01-07,430.209991
...,...
2023-06-16,408.690002
2023-06-20,403.489990
2023-06-21,402.309998


- Beware of the dates. We want to predict the next day, so we have to split the data as such. 
- `historical_data`: historical data to train
- `tomorrow`: the true label for the next day

In [4]:
historical_data = uts_observations[:-1]
historical_data

,Open
Date,
2022-01-03,437.929993
2022-01-04,440.579987
2022-01-05,438.779999
2022-01-06,430.200012
2022-01-07,430.209991
...,...
2023-06-15,401.000000
2023-06-16,408.690002
2023-06-20,403.489990


In [5]:
tomorrow = uts_observations[-1:]
tomorrow

,Open
Date,
2023-06-23,399.329987


In [6]:
tomorrow = tomorrow.values
tomorrow

array([[399.32998657]])

In [7]:
prior_observations, forecasting_step = [3, 1]
sml_samples_df = data_processor.convert_uts_sequence_to_sml_with_pd(historical_data, prior_observations, forecasting_step)
sml_samples_df

,t-3,t-2,t-1,t
0,437.929993,440.579987,438.779999,430.200012
1,440.579987,438.779999,430.200012,430.209991
2,438.779999,430.200012,430.209991,425.380005
3,430.200012,430.209991,425.380005,427.679993
4,430.209991,425.380005,427.679993,433.559998
...,...,...,...,...
361,395.980011,400.019989,401.609985,401.000000
362,400.019989,401.609985,401.000000,408.690002
363,401.609985,401.000000,408.690002,403.489990
364,401.000000,408.690002,403.489990,402.309998


- Matrix X shape is [samples, features] $ \Rightarrow $ [366, 3] $ \Rightarrow $ vector x shape is [1, 3]
- vector y shape is [samples, 1] $ \Rightarrow $ [366, 1] $ \Rightarrow $ scalar y shape is [1, 1]

In [8]:
X = sml_samples_df.iloc[:, 0:3]
X

,t-3,t-2,t-1
0,437.929993,440.579987,438.779999
1,440.579987,438.779999,430.200012
2,438.779999,430.200012,430.209991
3,430.200012,430.209991,425.380005
4,430.209991,425.380005,427.679993
...,...,...,...
361,395.980011,400.019989,401.609985
362,400.019989,401.609985,401.000000
363,401.609985,401.000000,408.690002
364,401.000000,408.690002,403.489990


In [9]:
y = sml_samples_df.iloc[0:, -1]
y

0      430.200012
1      430.209991
2      425.380005
3      427.679993
4      433.559998
          ...    
361    401.000000
362    408.690002
363    403.489990
364    402.309998
365    400.279999
Name: (t,), Length: 366, dtype: float64

In [10]:
# import os  
# os.makedirs('../datasets/', exist_ok=True)  
# y.to_csv('../datasets/y-voo.csv') 

# Machine Learning Models

In [11]:
@dataclass
class Model(ABC):
    """This is a base class with an abstract-like implementation. Each specified model inherits from this base class.
    
    Methods decorated with @abstractmethod must be implemented; if not, the interpreter will throw
    an error. Methods not decorated will be shared by all other classes that inherit from Model.
    """
    
    @abstractmethod
    def __name__(self):
        pass
    
    @abstractmethod
    def define_model_with_layers(self):
        pass
    
    @abstractmethod
    def fit_model(self):
        pass
    
    @abstractmethod
    def predict(self):
        pass
    
    def augment_data(self):
        pass

# MLP

In [12]:
class MLP(Model):
    
    def __name__(self):
        return "MLP"
    
    def define_model_with_layers(self, prior_observations, forecasting_step):
        """
        prior_observations -- int (of #cols; inputs; #nodes in 1st layer)
        forecasting_step -- int (of how far out we want to forecast)
        """
        model = Sequential()
        
        hidden_layer_1 = Dense(100, activation='relu', input_dim=prior_observations, name="hidden_layer")
        model.add(hidden_layer_1)
        input_weights_hidden = hidden_layer_1.get_weights()
        # print(len(input_weights_hidden), input_weights_hidden)
        # print(len(input_weights_hidden[0]))
        # print(len(input_weights_hidden[0][0]))
        # print(np.shape(input_weights_hidden[0][0]))

        output_layer = Dense(forecasting_step, name="output_layer")
        model.add(output_layer)
        hidden_weights_output = output_layer.get_weights()
        # print(len(hidden_weights_output), hidden_weights_output)
        # print(len(hidden_weights_output[0]))
        # print(len(hidden_weights_output[0][0]))
        # print(np.shape(hidden_weights_output[0][0]))
        
        model.compile(optimizer='adam', loss='mse')
        keras.utils.plot_model(model, "my_first_model.png", show_shapes=True)

        return model, model.summary()
    
    def fit_model(self, model_to_fit, X_train, y_train):
        model_to_fit.fit(X_train, y_train, epochs=2000, verbose=0)
        
        return model_to_fit
    
    def predict(self, fitted_model, x_input, prior_observations):
        """
        Parameters:
        fitted_model -- keras model
        x_input -- pd series
        prior_observations -- int
        
        Returns
        -- int (of the prediction at the next time step(s))
        """
        x_input = x_input.values
        x_input = x_input.reshape((1, prior_observations))
        yhat = fitted_model.predict(x_input)
        
        return yhat

In [13]:
mlp_model_class = MLP()

In [14]:
prior_observations

3

In [15]:
# !pip install pydot

In [16]:
mlp_model, model_defined = mlp_model_class.define_model_with_layers(prior_observations, forecasting_step)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer (Dense)        (None, 100)               400       
                                                                 
 output_layer (Dense)        (None, 1)                 101       
                                                                 
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________


2023-09-26 18:42:04.432259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


- Map each vector x sample to each scalar y true sample
- ie:
    1. sample 0 for vector x and scalar y $ \Rightarrow $ vector x [437.929993, 440.579987, 438.779999] maps to scalar y [430.200012]
    2. sample 1 for vector x and scalar y $ \Rightarrow $ vector x [440.579987, 438.779999, 430.200012] maps to scalar y [430.209991]

In [17]:
X

,t-3,t-2,t-1
0,437.929993,440.579987,438.779999
1,440.579987,438.779999,430.200012
2,438.779999,430.200012,430.209991
3,430.200012,430.209991,425.380005
4,430.209991,425.380005,427.679993
...,...,...,...
361,395.980011,400.019989,401.609985
362,400.019989,401.609985,401.000000
363,401.609985,401.000000,408.690002
364,401.000000,408.690002,403.489990


In [18]:
mlp_model_fitted = mlp_model_class.fit_model(mlp_model, X, y)

In [19]:
mlp_model_fitted

- Map the last 3 vector x samples (days) to the next scalar y true sample (day)
- ie:
    1. sample 1 for vector x and scalar y $ \Rightarrow $ vector x [403.489990, 402.309998, 400.279999] maps to scalar y [399.329987]

In [20]:
x_input = sml_samples_df.iloc[-1, -prior_observations:]
x_input

t-2    403.489990
t-1    402.309998
t      400.279999
Name: 365, dtype: float64

In [21]:
tomorrow

array([[399.32998657]])

In [22]:
mlp_model_prediction = mlp_model_class.predict(mlp_model_fitted, x_input, prior_observations)

1/1 [==============================] - 0s 60ms/step


In [23]:
mlp_model_prediction

array([[400.83237]], dtype=float32)

# CNN

In [24]:
class CNN(Model):
    
    def __name__(self):
        return "CNN"
    
    def define_model_with_layers(self, prior_observations, forecasting_step, n_features=1):
        model = Sequential()
        
        conv1d_layer = Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(prior_observations, n_features), name="conv1d_layer")
        model.add(conv1d_layer)
        input_weights_hidden = conv1d_layer.get_weights()
#         print(len(input_weights_hidden), input_weights_hidden)
#         print(len(input_weights_hidden[0]))
#         print(len(input_weights_hidden[0][0]))
#         print(np.shape(input_weights_hidden[0][0]))
        
        max_pooling_layer = MaxPooling1D(pool_size=2, strides=1, name="max_pooling_layer")
        model.add(max_pooling_layer)
        
        flatten_layer = Flatten(name="flatten_layer")
        model.add(flatten_layer)
        
        hidden_layer = Dense(50, activation='relu', name="hidden_layer")
        model.add(hidden_layer)
        
        output_layer = Dense(forecasting_step, name="output_layer")
        model.add(output_layer)
        
        model.compile(optimizer='adam', loss='mse')
        return model, model.summary()
    
    def fit_model(self, model_to_fit, X_train, y_train):
        model_to_fit.fit(X_train, y_train, epochs=2000, verbose=0)
        
        return model_to_fit
    
    def predict(self, fitted_model, x_input, prior_observations, n_features=1):
        """
        Parameters:
        fitted_model -- keras model
        x_input -- pd series
        prior_observations -- int
        
        Returns
        -- int (of the prediction at the next time step(s))
        """
        x_input = x_input.values
        x_input = x_input.reshape((1, prior_observations, n_features))
        yhat = fitted_model.predict(x_input)
        
        return yhat

In [25]:
cnn_model_class = CNN()

In [26]:
cnn_model, cnn_model_defined = cnn_model_class.define_model_with_layers(prior_observations, forecasting_step)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_layer (Conv1D)       (None, 2, 64)             192       
                                                                 
 max_pooling_layer (MaxPooli  (None, 1, 64)            0         
 ng1D)                                                           
                                                                 
 flatten_layer (Flatten)     (None, 64)                0         
                                                                 
 hidden_layer (Dense)        (None, 50)                3250      
                                                                 
 output_layer (Dense)        (None, 1)                 51        
                                                                 
Total params: 3,493
Trainable params: 3,493
Non-trainable params: 0
____________________________________________________

In [27]:
cnn_model_fitted = cnn_model_class.fit_model(cnn_model, X, y)

In [28]:
cnn_model_prediction = cnn_model_class.predict(cnn_model_fitted, x_input, prior_observations)

1/1 [==============================] - 0s 65ms/step


In [29]:
cnn_model_prediction

array([[400.69345]], dtype=float32)

In [30]:
class LSTM(Model):
    
    def __name__(self):
        return "LSTM"
    
    def define_model_with_layers(self, prior_observations, forecasting_step, n_features=1):
        model = Sequential()
        
        lstm_layer = model_LSTM(50, activation='relu', input_shape=(prior_observations, n_features))
        model.add(lstm_layer)
        input_weights_hidden = lstm_layer.get_weights()
        
        output_layer = Dense(forecasting_step, name="output_layer")
        model.add(output_layer)
        
        model.compile(optimizer='adam', loss='mse')
        return model, model.summary()
    
    def fit_model(self, model_to_fit, X_train, y_train):
        model_to_fit.fit(X_train, y_train, epochs=2000, verbose=0)
        
        return model_to_fit
    
    def predict(self, fitted_model, x_input, prior_observations, n_features=1):
        """
        Parameters:
        fitted_model -- keras model
        x_input -- pd series
        prior_observations -- int
        
        Returns
        -- int (of the prediction at the next time step(s))
        """
        x_input = x_input.values
        x_input = x_input.reshape((1, prior_observations, n_features))
        yhat = fitted_model.predict(x_input)
        
        return yhat

In [31]:
lstm_model_class = LSTM()
lstm_model_class

LSTM()

In [32]:
lstm_model, lstm_model_defined = lstm_model_class.define_model_with_layers(prior_observations, forecasting_step)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 output_layer (Dense)        (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [33]:
lstm_model_fitted = lstm_model_class.fit_model(lstm_model, X, y)

In [34]:
lstm_model_prediction = lstm_model_class.predict(lstm_model_fitted, x_input, prior_observations)

1/1 [==============================] - 0s 153ms/step


# Evaluation Metrics

- Points
    - true_tomorrow = [1, 2]
    - predicted_tomorrow = [3, 8]

---

1. MRE (max_error): Calculate the maximum residual error of points. Ex.:
    - $ \Rightarrow $ 3 - 1 = 2
    - $ \Rightarrow $ 8 - 2 = 6
    - $ \Rightarrow $ 6 > 2, so choose 6 as our MRE
    - Why use in the forecating stocks? We can know the greatest distance between our points. Knowing this will tell us our upper bound to know that our error of all the remaining points fall below this upper bound.
2. MSE (mean_squared_error): Calculate the average of the squared difference (or difference squared) between points. Ex.:
    - $ \Rightarrow $ 3 - 1 = 2 $ \Rightarrow $ $ 2^2 $ = 4
    - $ \Rightarrow $ 8 - 2 = 6 $ \Rightarrow $ $ 6^2 $ = 36
    - $ \Rightarrow $ 4 + 36 = 40
    - $ \Rightarrow $ 40 / 2 = 20, so choose 20 as our MSE
    - Why use in the forecating stocks? We can know the average distance between our points. Knowing this error provides insight to how spread our points are.
3. MAE (mean_absolute_error): Use to 
    - Why use in the forecating stocks?
4. MAPE (mean_absolute_percentage_error): Use to 
    - Why use in the forecating stocks?


In [35]:
# Can ignore; only here to ensure calculations of the errors are correct
true_tomorrow = [1, 2]
predicted_tomorrow = [3, 8]

print(max_error(true_tomorrow, predicted_tomorrow))
print(mean_squared_error(true_tomorrow, predicted_tomorrow))

6
20.0


In [36]:
@dataclass
class EvaluationMetric:
    """Investigate the philosphy/design behind typing in python. 
    
    https://realpython.com/python-type-checking/
    https://machinelearningmastery.com/regression-metrics-for-machine-learning/
    """
    models: List[Model]
    
    def __post_init__(self):
        pass
    
    def _get_eval_handles(self):
        """dir(self) grabs all of the available methods and attributes on the class itself."""
        function_names = [name for name in dir(self) if callable(getattr(self, name)) and not name.startswith("__")]
        # print("All functions in this class: ", function_names, "\n")
        
        # Filter for metrics that follow our specific specification and return their names
        function_names = [fn for fn in function_names if fn[:5] == "eval_"]
        # print("Functions that start with eval_:", function_names, "\n")
        
        # Get a handle on the function objects themselves
        functions = [getattr(self, name) for name in function_names]
        # print("Store functions that start with eval_:", functions)
        
        return functions
    
    def perform_evaluations(self, tomorrow, model_prediction):
        eval_metrics = self._get_eval_handles()
        eval_results = {}
        
        # eval_metrics = [eval_mse, eval_mape, ...] --> these are actual function handles we can
        # iterate over
        # print(eval_metrics)
        for eval_metric in eval_metrics:
            # print("Evaluation Metric", eval_metric)
            eval_results[eval_metric.__name__] = eval_metric(tomorrow, model_prediction) # eval_metric() == self.eval_mse()
        # print(eval_results)
        
        # Update the state of our object
        self.eval_results = eval_results
        return eval_results
    
    def eval_mre(self, true_tomorrow, model_prediction_tomorrow) -> float:
        return {'MRE': max_error(true_tomorrow, model_prediction_tomorrow)}
    
    def eval_mse(self, true_tomorrow, model_prediction_tomorrow) -> float:
        return {'MSE': mean_squared_error(true_tomorrow, model_prediction_tomorrow)}
    
    def eval_mae(self, true_tomorrow, model_prediction_tomorrow) -> float:
        return {'MAE': mean_absolute_error(true_tomorrow, model_prediction_tomorrow)}
    
    def eval_mape(self, true_tomorrow, model_prediction_tomorrow) -> float:
        return {'MAPE': mean_absolute_percentage_error(true_tomorrow, model_prediction_tomorrow)}

In [37]:
models = [mlp_model_class, cnn_model_class, lstm_model_class]
eval_metrics = EvaluationMetric(models)

In [38]:
models

[MLP(), CNN(), LSTM()]

In [39]:
# eval_metrics.models

In [40]:
# eval_metrics_models = eval_metrics.models

In [41]:
mlp_model_prediction

array([[400.83237]], dtype=float32)

In [42]:
mlp_eval_metrics = eval_metrics.perform_evaluations(tomorrow, mlp_model_prediction)
mlp_eval_metrics

{'eval_mae': {'MAE': 1.50238037109375},
 'eval_mape': {'MAPE': 0.003762252827516795},
 'eval_mre': {'MRE': 1.50238037109375},
 'eval_mse': {'MSE': 2.257146779447794}}

In [43]:
cnn_model_prediction

array([[400.69345]], dtype=float32)

In [44]:
cnn_eval_metrics = eval_metrics.perform_evaluations(tomorrow, cnn_model_prediction)
cnn_eval_metrics

{'eval_mae': {'MAE': 1.36346435546875},
 'eval_mape': {'MAPE': 0.0034143800899409985},
 'eval_mre': {'MRE': 1.36346435546875},
 'eval_mse': {'MSE': 1.8590350486338139}}

In [45]:
lstm_eval_metrics = eval_metrics.perform_evaluations(tomorrow, lstm_model_prediction)
lstm_eval_metrics

{'eval_mae': {'MAE': 1.003692626953125},
 'eval_mape': {'MAPE': 0.0025134416665488495},
 'eval_mre': {'MRE': 1.003692626953125},
 'eval_mse': {'MSE': 1.007398889400065}}

In [46]:
metrics = [mlp_eval_metrics, cnn_eval_metrics, lstm_eval_metrics]

In [47]:
@dataclass
class EvaluationResults:
    """Our diagnostic or reporter class to be further implemented if desired."""
    # eval_metric: EvaluationMetric
    
    def summarize(self, eval_metrics_models, metrics):
        store_eval_metrics = {}
        
        for eval_metrics_models_idx in range(len(eval_metrics_models)):
            model = str(eval_metrics_models[eval_metrics_models_idx])
            store_eval_metrics[model] = metrics[eval_metrics_models_idx]
        return store_eval_metrics

In [48]:
eval_results = EvaluationResults()

In [49]:
eval_results

EvaluationResults()

In [50]:
eval_results.summarize(models, metrics)

{'MLP()': {'eval_mae': {'MAE': 1.50238037109375},
  'eval_mape': {'MAPE': 0.003762252827516795},
  'eval_mre': {'MRE': 1.50238037109375},
  'eval_mse': {'MSE': 2.257146779447794}},
 'CNN()': {'eval_mae': {'MAE': 1.36346435546875},
  'eval_mape': {'MAPE': 0.0034143800899409985},
  'eval_mre': {'MRE': 1.36346435546875},
  'eval_mse': {'MSE': 1.8590350486338139}},
 'LSTM()': {'eval_mae': {'MAE': 1.003692626953125},
  'eval_mape': {'MAPE': 0.0025134416665488495},
  'eval_mre': {'MRE': 1.003692626953125},
  'eval_mse': {'MSE': 1.007398889400065}}}